In [1]:
import sys

sys.path.append("../")

In [4]:
from torchvision import transforms as T
from src.data.functional import get_kfold_ctssev

transform = T.ToTensor()
train_dataset, test_dataset = get_kfold_ctssev(
  data_path="../data",
  num_splits=5,
  num_repeats=10,
  nth_fold=19,
  train_transform=transform,
  eval_transform=transform,
)

In [18]:
import torch

values = []
for data in train_dataset:
  values.append(data["inputs1"].view(1, -1))
  values.append(data["inputs2"].view(1, -1))
values = torch.cat(values, dim=0)
values.size()

torch.Size([1058, 433758])

In [19]:
print("Mean:", torch.mean(values))
print("Std:", torch.std(values))

Mean: tensor(0.1651)
Std: tensor(0.1553)


In [6]:
severities = [test_dataset.dataset.severities[i] for i in test_dataset.indices]
for severity in ["normal", "mild", "moderate", "severe"]:
  print(f"{severity.capitalize()}:", severities.count(severity))

Normal: 43
Mild: 21
Moderate: 12
Severe: 22


In [11]:
from src.data.functional import get_kfold_ctsdiag

train_dataset, test_dataset = get_kfold_ctsdiag(
  data_path="../data",
  num_splits=5,
  num_repeats=10,
  nth_fold=19,
  train_transform=transform,
  eval_transform=transform,
)
labels = [test_dataset.dataset.labels[i] for i in test_dataset.indices]
print("Normal:", labels.count(0))
print("Abnormal:", labels.count(1))

Normal: 43
Abnormal: 55


In [8]:
from src.data import CTSDiag

dataset = CTSDiag(root="../data", split=None)

NORMAL, MILD, MODERATE, SEVERE = 0, 1, 2, 3
SINGLE_VP, MULTIPLE_VP = 0, 1
data = [[[], []], [[], []], [[], []], [[], []]]
for patient, severity in zip(dataset.patients, dataset.severities):
  severity = ["normal", "mild", "moderate", "severe"].index(severity)
  is_single = dataset.patients.count(patient) == 1
  data[severity][SINGLE_VP if is_single else MULTIPLE_VP].append(patient)

for severity in ["normal", "mild", "moderate", "severe"]:
  print(severity.capitalize())
  severity = ["normal", "mild", "moderate", "severe"].index(severity)
  print(f"  single viewpoint: {len(set(data[severity][0]))} / {len(data[severity][0])}")
  print(f"  multiple viewpoints: {len(set(data[severity][1]))} / {len(data[severity][1])}")

sum(map(lambda e: len(e[0]) + len(e[1]), data))

Normal
  single viewpoint: 16 / 16
  multiple viewpoints: 24 / 244
Mild
  single viewpoint: 9 / 9
  multiple viewpoints: 9 / 90
Moderate
  single viewpoint: 17 / 17
  multiple viewpoints: 12 / 120
Severe
  single viewpoint: 11 / 11
  multiple viewpoints: 12 / 120


627